<a href="https://colab.research.google.com/github/Sujitdesai/Data_Science_Projects/blob/main/Heisenberg_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')


In [ ]:
df_train.drop(df_train[df_train['Duration'] > 550].index,inplace= True)
df_train.drop(df_train[df_train['Duration'] < 0].index,inplace= True)
#df_train.drop(df_train[df_train['Net Sales'] < 0].index,inplace= True)
df_train.drop(df_train[df_train['Age'] <= 0].index,inplace= True)
#df_train.drop(df_train[df_train['Age'] >= 90].index,inplace= True)


In [ ]:
df_train['Claim'].value_counts()

0    43574
1     8720
Name: Claim, dtype: int64

In [ ]:
df_train = pd.get_dummies(df_train,columns=['Agency','Agency Type','Distribution Channel','Product Name','Destination'],drop_first=True)
df_train.head()

,ID,Duration,Net Sales,Commision (in value),Age,Claim,Agency_ART,Agency_C2B,Agency_CBH,Agency_CCR,Agency_CSR,Agency_CWT,Agency_EPX,Agency_JWT,Agency_JZI,Agency_KML,Agency_LWC,Agency_RAB,Agency_SSI,Agency_TST,Agency_TTW,Agency Type_Travel Agency,Distribution Channel_Online,Product Name_2 way Comprehensive Plan,Product Name_24 Protect,Product Name_Annual Gold Plan,Product Name_Annual Silver Plan,Product Name_Annual Travel Protect Gold,Product Name_Annual Travel Protect Platinum,Product Name_Annual Travel Protect Silver,Product Name_Basic Plan,Product Name_Bronze Plan,Product Name_Cancellation Plan,Product Name_Child Comprehensive Plan,Product Name_Comprehensive Plan,Product Name_Gold Plan,Product Name_Individual Comprehensive Plan,Product Name_Premier Plan,Product Name_Rental Vehicle Excess Insurance,Product Name_Silver Plan,...,Destination_MEXICO,Destination_MONGOLIA,Destination_MOROCCO,Destination_MYANMAR,Destination_NEPAL,Destination_NETHERLANDS,Destination_NEW ZEALAND,Destination_NORWAY,Destination_OMAN,Destination_PAKISTAN,Destination_PERU,Destination_PHILIPPINES,Destination_POLAND,Destination_PORTUGAL,Destination_QATAR,Destination_ROMANIA,Destination_RUSSIAN FEDERATION,Destination_SAUDI ARABIA,Destination_SINGAPORE,Destination_SLOVENIA,Destination_SOUTH AFRICA,Destination_SPAIN,Destination_SRI LANKA,Destination_SWEDEN,Destination_SWITZERLAND,"Destination_TAIWAN, PROVINCE OF CHINA","Destination_TANZANIA, UNITED REPUBLIC OF",Destination_THAILAND,Destination_TUNISIA,Destination_TURKEY,Destination_TURKMENISTAN,Destination_UKRAINE,Destination_UNITED ARAB EMIRATES,Destination_UNITED KINGDOM,Destination_UNITED STATES,Destination_UZBEKISTAN,Destination_VANUATU,Destination_VENEZUELA,Destination_VIET NAM,Destination_ZAMBIA
0,2010,61,12.0,0.00,41,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4245,4,17.0,0.00,35,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,9251,26,19.8,11.88,47,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,4754,15,27.0,0.00,48,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8840,15,37.0,0.00,36,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# RobustScaler is less prone to outliers.
mm_scaler = MinMaxScaler()

df_train['scaled_duration'] = mm_scaler.fit_transform(df_train['Duration'].values.reshape(-1,1))
df_train['scaled_Net_Sales'] = mm_scaler.fit_transform(df_train['Net Sales'].values.reshape(-1,1))
df_train['scaled_Commision'] = mm_scaler.fit_transform(df_train['Commision (in value)'].values.reshape(-1,1))
df_train['scaled_age'] = mm_scaler.fit_transform(df_train['Age'].values.reshape(-1,1))

df_train.drop(['Duration','Net Sales','Commision (in value)','Age'], axis=1, inplace=True)
scaled_duration = df_train['scaled_duration']
scaled_Net_Sales = df_train['scaled_Net_Sales']
scaled_Commision=df_train['scaled_Commision']
scaled_age = df_train['scaled_age']

df_train.drop(['scaled_duration', 'scaled_Net_Sales','scaled_Commision','scaled_age'], axis=1, inplace=True)
df_train.insert(0, 'scaled_duration', scaled_duration)
df_train.insert(1, 'scaled_Net_Sales', scaled_Net_Sales)
df_train.insert(2,'scaled_Commision',scaled_Commision)
df_train.insert(3, 'scaled_age', scaled_age)


df_train.head()


,scaled_duration,scaled_Net_Sales,scaled_Commision,scaled_age,ID,Claim,Agency_ART,Agency_C2B,Agency_CBH,Agency_CCR,Agency_CSR,Agency_CWT,Agency_EPX,Agency_JWT,Agency_JZI,Agency_KML,Agency_LWC,Agency_RAB,Agency_SSI,Agency_TST,Agency_TTW,Agency Type_Travel Agency,Distribution Channel_Online,Product Name_2 way Comprehensive Plan,Product Name_24 Protect,Product Name_Annual Gold Plan,Product Name_Annual Silver Plan,Product Name_Annual Travel Protect Gold,Product Name_Annual Travel Protect Platinum,Product Name_Annual Travel Protect Silver,Product Name_Basic Plan,Product Name_Bronze Plan,Product Name_Cancellation Plan,Product Name_Child Comprehensive Plan,Product Name_Comprehensive Plan,Product Name_Gold Plan,Product Name_Individual Comprehensive Plan,Product Name_Premier Plan,Product Name_Rental Vehicle Excess Insurance,Product Name_Silver Plan,...,Destination_MEXICO,Destination_MONGOLIA,Destination_MOROCCO,Destination_MYANMAR,Destination_NEPAL,Destination_NETHERLANDS,Destination_NEW ZEALAND,Destination_NORWAY,Destination_OMAN,Destination_PAKISTAN,Destination_PERU,Destination_PHILIPPINES,Destination_POLAND,Destination_PORTUGAL,Destination_QATAR,Destination_ROMANIA,Destination_RUSSIAN FEDERATION,Destination_SAUDI ARABIA,Destination_SINGAPORE,Destination_SLOVENIA,Destination_SOUTH AFRICA,Destination_SPAIN,Destination_SRI LANKA,Destination_SWEDEN,Destination_SWITZERLAND,"Destination_TAIWAN, PROVINCE OF CHINA","Destination_TANZANIA, UNITED REPUBLIC OF",Destination_THAILAND,Destination_TUNISIA,Destination_TURKEY,Destination_TURKMENISTAN,Destination_UKRAINE,Destination_UNITED ARAB EMIRATES,Destination_UNITED KINGDOM,Destination_UNITED STATES,Destination_UZBEKISTAN,Destination_VANUATU,Destination_VENEZUELA,Destination_VIET NAM,Destination_ZAMBIA
0,0.111517,0.374416,0.000000,0.341880,2010,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.007313,0.379085,0.000000,0.290598,4245,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.047532,0.381699,0.045212,0.393162,9251,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0.027422,0.388422,0.000000,0.401709,4754,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.027422,0.397759,0.000000,0.299145,8840,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler


mm_scaler = MinMaxScaler()


df_test['scaled_duration'] = mm_scaler.fit_transform(df_test['Duration'].values.reshape(-1,1))
df_test['scaled_Net_Sales'] = mm_scaler.fit_transform(df_test['Net Sales'].values.reshape(-1,1))
df_test['scaled_Commision'] = mm_scaler.fit_transform(df_test['Commision (in value)'].values.reshape(-1,1))
df_test['scaled_age'] = mm_scaler.fit_transform(df_test['Age'].values.reshape(-1,1))

df_test.drop(['Duration','Net Sales','Commision (in value)','Age'], axis=1, inplace=True)
scaled_duration = df_test['scaled_duration']
scaled_Net_Sales = df_test['scaled_Net_Sales']
scaled_Commision=df_test['scaled_Commision']
scaled_age = df_test['scaled_age']

df_test.drop(['scaled_duration', 'scaled_Net_Sales','scaled_Commision','scaled_age'], axis=1, inplace=True)
df_test.insert(0, 'scaled_duration', scaled_duration)
df_test.insert(1, 'scaled_Net_Sales', scaled_Net_Sales)
df_test.insert(2,'scaled_Commision',scaled_Commision)
df_test.insert(3, 'scaled_age', scaled_age)


df_test.head()


,scaled_duration,scaled_Net_Sales,scaled_Commision,scaled_age,ID,Agency,Agency Type,Distribution Channel,Product Name,Destination
0,0.039728,0.284553,0.000000,0.299145,17631,EPX,Travel Agency,Online,Cancellation Plan,HONG KONG
1,0.000618,0.286360,0.000000,0.299145,15064,EPX,Travel Agency,Online,1 way Comprehensive Plan,SINGAPORE
2,0.002882,0.280488,0.011922,0.196581,14139,C2B,Airlines,Online,Bronze Plan,SINGAPORE
3,0.027583,0.305330,0.000000,0.299145,19754,EPX,Travel Agency,Online,2 way Comprehensive Plan,"TAIWAN, PROVINCE OF CHINA"
4,0.000618,0.295393,0.026455,0.264957,16439,C2B,Airlines,Online,Silver Plan,SINGAPORE


In [ ]:
# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
import collections


# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

In [ ]:

df_train=df_train.drop(columns=['ID'])
X=df_train.drop(columns=['Claim'])
y=df_train['Claim']



In [ ]:
from imblearn.over_sampling import RandomOverSampler

ors=RandomOverSampler(ratio=1)
X_train_res,y_train_res=ors.fit_sample(X,y)

In [ ]:
X_train_res.shape,y_train_res.shape

((87148, 141), (87148,))

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_train_res,y_train_res,test_size=0.30,random_state=101)

In [ ]:
db = AdaBoostClassifier(RandomForestClassifier(),n_estimators=100,learning_rate=1)
db.fit(X_train,y_train)
db.score(X_test,y_test)

0.9780837636259323

In [ ]:
prediction = db.predict(X_test)
prec1 = precision_score(y_test,prediction)
prec1

0.9627439384979303

In [ ]:
etc=ExtraTreesClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=False, oob_score=False, n_jobs=-1, random_state=101, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
etc.fit(X_train,y_train)
etc.score(X_test,y_test)

0.9747944157582712

In [ ]:
prediction = etc.predict(X_test)
prec1 = precision_score(y_test,prediction)

prec1

0.9569307658386006

In [ ]:
df_test = pd.get_dummies(df_test,columns=['Agency','Agency Type','Distribution Channel','Product Name','Destination'],drop_first=True)
df_test.head()

,scaled_duration,scaled_Net_Sales,scaled_Commision,scaled_age,ID,Agency_ART,Agency_C2B,Agency_CBH,Agency_CCR,Agency_CSR,Agency_CWT,Agency_EPX,Agency_JWT,Agency_JZI,Agency_KML,Agency_LWC,Agency_RAB,Agency_SSI,Agency_TST,Agency_TTW,Agency Type_Travel Agency,Distribution Channel_Online,Product Name_2 way Comprehensive Plan,Product Name_24 Protect,Product Name_Annual Gold Plan,Product Name_Annual Silver Plan,Product Name_Annual Travel Protect Gold,Product Name_Annual Travel Protect Platinum,Product Name_Annual Travel Protect Silver,Product Name_Basic Plan,Product Name_Bronze Plan,Product Name_Cancellation Plan,Product Name_Child Comprehensive Plan,Product Name_Comprehensive Plan,Product Name_Gold Plan,Product Name_Individual Comprehensive Plan,Product Name_Premier Plan,Product Name_Rental Vehicle Excess Insurance,Product Name_Silver Plan,Product Name_Single Trip Travel Protect Gold,...,Destination_MEXICO,Destination_MONGOLIA,Destination_MOROCCO,Destination_MYANMAR,Destination_NEPAL,Destination_NETHERLANDS,Destination_NEW ZEALAND,Destination_NORWAY,Destination_OMAN,Destination_PAKISTAN,Destination_PERU,Destination_PHILIPPINES,Destination_POLAND,Destination_PORTUGAL,Destination_QATAR,Destination_ROMANIA,Destination_RUSSIAN FEDERATION,Destination_SAUDI ARABIA,Destination_SINGAPORE,Destination_SLOVENIA,Destination_SOUTH AFRICA,Destination_SPAIN,Destination_SRI LANKA,Destination_SWEDEN,Destination_SWITZERLAND,"Destination_TAIWAN, PROVINCE OF CHINA","Destination_TANZANIA, UNITED REPUBLIC OF",Destination_THAILAND,Destination_TUNISIA,Destination_TURKEY,Destination_TURKMENISTAN,Destination_UKRAINE,Destination_UNITED ARAB EMIRATES,Destination_UNITED KINGDOM,Destination_UNITED STATES,Destination_UZBEKISTAN,Destination_VANUATU,Destination_VENEZUELA,Destination_VIET NAM,Destination_ZAMBIA
0,0.039728,0.284553,0.000000,0.299145,17631,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.000618,0.286360,0.000000,0.299145,15064,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.002882,0.280488,0.011922,0.196581,14139,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.027583,0.305330,0.000000,0.299145,19754,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.000618,0.295393,0.026455,0.264957,16439,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X1_test=df_test.drop(columns=['ID'])
predict = etc.predict(X1_test)
predict

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
df_test['Claim']=pd.DataFrame(predict)

In [ ]:
df_test['Claim'].value_counts()

0    22047
1      374
Name: Claim, dtype: int64

In [ ]:
output=df_test[['ID','Claim']]
output.to_csv('Heisenberg_071.csv', index=False)